In [ ]:
#@title Import Libraries

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.models import model_from_json
from keras.models import load_model
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Embedding
from keras.layers.advanced_activations import ELU
from keras.layers import Dense, Bidirectional, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout
from keras.layers import LSTM, Flatten, Conv1D, LocallyConnected1D, GRU, Layer, MultiHeadAttention, LayerNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from keras import backend as K
import keras
import tensorflow as tf
import numpy as np
import argparse
import os
import sys
import json
import contextlib
import struct
import tempfile
import shutil
from math import sqrt
from tqdm.notebook import tqdm
from tensorflow.keras.utils import Progbar

In [ ]:
#@title GPU Initialization
#Execute only if using GPU

# strategy = tf.distribute.MirroredStrategy()

In [ ]:
#@title TPU Initialization
# Execute only if using TPU

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base = 'HP1'

with open('/content/drive/MyDrive/DeepZip/' + 'data/files_to_be_compressed/' + base + '.txt', 'rb') as fp:
    data = fp.read()

clean_data = ""
for character in data:
    if character < 128:
        clean_data += chr(character)

with open('/content/drive/MyDrive/DeepZip/' + 'data/files_to_be_compressed/' + base + '_clean.txt', 'w') as fp:
    fp.write(clean_data)

In [ ]:
#@title Data Preparation, Choosing Input File and Model
#data preparation - parse_new.py

base_path = '/content/drive/MyDrive/DeepZip/'
base_name = 'HP1_clean'
model_name = 'TransformerLarge_128'

input_file_path = base_path + 'data/files_to_be_compressed/' + base_name + '.txt'
param_file_path = base_path + 'data/processed_files/' + base_name + '.param.json'
output_file_path = base_path + 'data/processed_files/' + base_name + '.npy'

with open(input_file_path, 'rb') as fp:
    data = fp.read()

print(len(data))
vals = list(set(data))

print(len(vals))

char2id_dict = {c: i for (i,c) in enumerate(vals)}
id2char_dict = {i: c for (i,c) in enumerate(vals)}

params = {'char2id_dict':char2id_dict, 'id2char_dict':id2char_dict}
with open(param_file_path, 'w') as f:
    json.dump(params, f, indent=4)

print(char2id_dict)
print(id2char_dict)

out = [char2id_dict[c] for c in data]
integer_encoded = np.array(out)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
print(integer_encoded[:10])
print(data[:10])

np.save(output_file_path, integer_encoded)

In [ ]:
conversion_matrix = [[0]*128 for i in range(len(vals))]

for i in range(len(vals)):
    conversion_matrix[i][vals[i]] = 1

conversion_matrix = np.array(conversion_matrix).astype('uint8')

print(conversion_matrix.shape)

(77, 128)


In [ ]:
#@title Arithmetic Encoder

class ArithmeticCoderBase(object):
	
	def __init__(self, statesize):
		self.STATE_SIZE = statesize
		self.MAX_RANGE = 1 << self.STATE_SIZE
		self.MIN_RANGE = (self.MAX_RANGE >> 2) + 2
		self.MAX_TOTAL = self.MIN_RANGE
		self.MASK = self.MAX_RANGE - 1
		self.TOP_MASK = self.MAX_RANGE >> 1
		self.SECOND_MASK = self.TOP_MASK >> 1
		self.low = 0
		self.high = self.MASK
	
	def update(self,  cumul, symbol):
		low = self.low
		high = self.high

		range = high - low + 1

		total = np.asscalar(cumul[-1])
		symlow = np.asscalar(cumul[symbol])
		symhigh = np.asscalar(cumul[symbol+1])

		newlow  = low + symlow  * range // total
		newhigh = low + symhigh * range // total - 1
		self.low = newlow
		self.high = newhigh
		
		while ((self.low ^ self.high) & self.TOP_MASK) == 0:
			self.shift()
			self.low = (self.low << 1) & self.MASK
			self.high = ((self.high << 1) & self.MASK) | 1
		
		while (self.low & ~self.high & self.SECOND_MASK) != 0:
			self.underflow()
			self.low = (self.low << 1) & (self.MASK >> 1)
			self.high = ((self.high << 1) & (self.MASK >> 1)) | self.TOP_MASK | 1
	
	def shift(self):
		raise NotImplementedError()
	
	def underflow(self):
		raise NotImplementedError()

class ArithmeticEncoder(ArithmeticCoderBase):
	
	def __init__(self, statesize, bitout):
		super(ArithmeticEncoder, self).__init__(statesize)
		self.output = bitout
		self.num_underflow = 0
	
	def write(self, cumul, symbol):
		self.update(cumul, symbol)
	
	def finish(self):
		self.output.write(1)
	
	def shift(self):
		bit = self.low >> (self.STATE_SIZE - 1)
		self.output.write(bit)
		for _ in range(self.num_underflow):
			self.output.write(bit ^ 1)
		self.num_underflow = 0
	
	def underflow(self):
		self.num_underflow += 1

class ArithmeticDecoder(ArithmeticCoderBase):
	
	def __init__(self, statesize, bitin):
		super(ArithmeticDecoder, self).__init__(statesize)
		self.input = bitin
		self.code = 0
		for _ in range(self.STATE_SIZE):
			self.code = self.code << 1 | self.read_code_bit()
	
	def read(self, cumul, alphabet_size):
		total = np.asscalar(cumul[-1])
		range = self.high - self.low + 1
		offset = self.code - self.low
		value = ((offset + 1) * total - 1) // range
		start = 0
		end = alphabet_size
		while end - start > 1:
			middle = (start + end) >> 1
			if cumul[middle] > value:
				end = middle
			else:
				start = middle
		
		symbol = start
		self.update(cumul, symbol)
		return symbol
	
	def shift(self):
		self.code = ((self.code << 1) & self.MASK) | self.read_code_bit()
	
	def underflow(self):
		self.code = (self.code & self.TOP_MASK) | ((self.code << 1) & (self.MASK >> 1)) | self.read_code_bit()
	
	def read_code_bit(self):
		temp = self.input.read()
		if temp == -1:
			temp = 0
		return temp


class BitInputStream(object):
	
	def __init__(self, inp):
		self.input = inp
		self.currentbyte = 0
		self.numbitsremaining = 0
	
	def read(self):
		if self.currentbyte == -1:
			return -1
		if self.numbitsremaining == 0:
			temp = self.input.read(1)
			if len(temp) == 0:
				self.currentbyte = -1
				return -1
			self.currentbyte = temp[0]
			self.numbitsremaining = 8
		assert self.numbitsremaining > 0
		self.numbitsremaining -= 1
		return (self.currentbyte >> self.numbitsremaining) & 1
	
	def read_no_eof(self):
		result = self.read()
		if result != -1:
			return result
		else:
			raise EOFError()
	
	def close(self):
		self.input.close()
		self.currentbyte = -1
		self.numbitsremaining = 0


class BitOutputStream(object):
	
	def __init__(self, out):
		self.output = out
		self.currentbyte = 0
		self.numbitsfilled = 0
	
	def write(self, b):
		if b not in (0, 1):
			raise ValueError("Argument must be 0 or 1")
		self.currentbyte = (self.currentbyte << 1) | b
		self.numbitsfilled += 1
		if self.numbitsfilled == 8:
			towrite = bytes((self.currentbyte,))
			self.output.write(towrite)
			self.currentbyte = 0
			self.numbitsfilled = 0
	
	def close(self):
		while self.numbitsfilled != 0:
			self.write(0)
		self.output.close()

In [ ]:
#@title Models

class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class Models():

    def biGRU(bs,time_steps,alphabet_size):
        model = Sequential()
        model.add(Embedding(alphabet_size, 32))
        model.add(Bidirectional(GRU(32, stateful=False, return_sequences=True)))
        model.add(Bidirectional(GRU(32, stateful=False, return_sequences=False)))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(alphabet_size, activation='softmax'))
        return model

    # def biGRU_big(bs,time_steps,alphabet_size):
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(Bidirectional(GRU(128, stateful=False, return_sequences=True)))
    #     model.add(Bidirectional(GRU(128, stateful=False, return_sequences=False)))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def biGRU_16bit(bs,time_steps,alphabet_size):
    #     K.set_floatx('float16')
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(Bidirectional(GRU(32, stateful=False, return_sequences=True)))
    #     model.add(Bidirectional(GRU(32, stateful=False, return_sequences=False)))
    #     model.add(Dense(64, activation='relu'))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def biLSTM(bs,time_steps,alphabet_size):
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(Bidirectional(LSTM(32, stateful=False, return_sequences=True)))
    #     model.add(Bidirectional(LSTM(32, stateful=False, return_sequences=False)))
    #     model.add(Dense(64, activation='relu'))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def biLSTM_16bit(bs,time_steps,alphabet_size):
    #     K.set_floatx('float16')
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(Bidirectional(LSTM(32, stateful=False, return_sequences=True)))
    #     model.add(Bidirectional(LSTM(32, stateful=False, return_sequences=False)))
    #     model.add(Dense(64, activation='relu'))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def LSTM_multi(bs,time_steps,alphabet_size):
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(LSTM(32, stateful=False, return_sequences=True))
    #     model.add(LSTM(32, stateful=False, return_sequences=True))
    #     model.add(Flatten())
    #     model.add(Dense(64, activation='relu'))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def LSTM_multi_big(bs,time_steps,alphabet_size):
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 64, batch_input_shape=(bs, time_steps)))
    #     model.add(LSTM(64, stateful=False, return_sequences=True))
    #     model.add(LSTM(64, stateful=False, return_sequences=True))
    #     model.add(Flatten())
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def LSTM_multi_bn(bs,time_steps,alphabet_size):
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(LSTM(32, stateful=False, return_sequences=True))
    #     model.add(LSTM(32, stateful=False, return_sequences=True))
    #     model.add(Flatten())
    #     model.add(BatchNormalization())
    #     model.add(Dense(64, activation='relu'))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def LSTM_multi_16bit(bs,time_steps,alphabet_size):
    #     K.set_floatx('float16')
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(LSTM(32, stateful=False, return_sequences=True))
    #     model.add(LSTM(32, stateful=False, return_sequences=True))
    #     model.add(Flatten())
    #     model.add(Dense(64, activation='relu'))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def LSTM_multi_selu(bs,time_steps,alphabet_size):
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(LSTM(32, stateful=False, return_sequences=True))
    #     model.add(LSTM(32, stateful=False, return_sequences=True))
    #     model.add(Flatten())
    #     model.add(Dense(64, activation=keras.activations.selu, kernel_initializer=init))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def LSTM_multi_selu_16bit(bs,time_steps,alphabet_size):
    #     K.set_floatx('float16')
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(LSTM(32, stateful=False, return_sequences=True))
    #     model.add(LSTM(32, stateful=False, return_sequences=True))
    #     model.add(Flatten())
    #     init = keras.initializers.lecun_uniform(seed=0)
    #     model.add(Dense(64, activation=keras.activations.selu, kernel_initializer=init))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def GRU_multi(bs,time_steps,alphabet_size):
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(GRU(32, stateful=False, return_sequences=True))
    #     model.add(GRU(32, stateful=False, return_sequences=True))
    #     model.add(Flatten())
    #     model.add(Dense(64, activation='relu'))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def GRU_multi_big(bs,time_steps,alphabet_size):
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(GRU(128, stateful=False, return_sequences=True))
    #     model.add(GRU(128, stateful=False, return_sequences=True))
    #     model.add(Flatten())
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def GRU_multi_16bit(bs,time_steps,alphabet_size):
    #     K.set_floatx('float16')
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(GRU(32, stateful=False, return_sequences=True))
    #     model.add(GRU(32, stateful=False, return_sequences=True))
    #     model.add(Flatten())
    #     model.add(Dense(64, activation='relu'))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def FC_4layer_16bit(bs,time_steps, alphabet_size):
    #     K.set_floatx('float16')
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 5, batch_input_shape=(bs, time_steps)))
    #     model.add(Flatten())
    #     model.add(Dense(128, activation=ELU(1.0)))
    #     model.add(Dense(128, activation=ELU(1.0)))
    #     model.add(Dense(128, activation=ELU(1.0)))
    #     model.add(Dense(128, activation=ELU(1.0)))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def FC_4layer(bs,time_steps, alphabet_size):
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 5, batch_input_shape=(bs, time_steps)))
    #     model.add(Flatten())
    #     model.add(Dense(128, activation=ELU(1.0)))
    #     model.add(Dense(128, activation=ELU(1.0)))
    #     model.add(Dense(128, activation=ELU(1.0)))
    #     model.add(Dense(128, activation=ELU(1.0)))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def FC_4layer_big(bs,time_steps, alphabet_size):
    #     model = Sequential()
    #     model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(Flatten())
    #     model.add(Dense(128, activation=ELU(1.0)))
    #     model.add(Dense(128, activation=ELU(1.0)))
    #     model.add(Dense(128, activation=ELU(1.0)))
    #     model.add(Dense(128, activation=ELU(1.0)))
    #     model.add(Dense(alphabet_size, activation='softmax'))
    #     return model

    # def FC_16bit(bs,time_steps,alphabet_size):
    #     k.set_floatx('float16')
    #     model = Sequential()
    #     init = keras.initializers.lecun_uniform(seed=0)
    #     model.add(embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
    #     model.add(flatten())
    #     model.add(dense(1024, activation='relu', kernel_initializer=init))
    #     model.add(dense(64, activation='relu', kernel_initializer=init))
    #     model.add(dense(alphabet_size, activation='softmax'))
    #     return model

    def FC(bs,time_steps,alphabet_size):
        model = Sequential()
        init = keras.initializers.lecun_uniform(seed=0)
        model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
        model.add(Flatten())
        model.add(Dense(1024, activation='relu', kernel_initializer=init))
        model.add(Dense(64, activation='relu', kernel_initializer=init))
        model.add(Dense(alphabet_size, activation='softmax'))
        return model

    def Transformer(bs, time_steps, alphabet_size):
        model = Sequential()
        model.add(TokenAndPositionEmbedding(time_steps, alphabet_size, 32))
        model.add(TransformerBlock(32, 2, 32))
        model.add(TransformerBlock(32, 2, 32))
        model.add(TransformerBlock(32, 2, 32))
        model.add(TransformerBlock(32, 2, 32))
        model.add(TransformerBlock(32, 2, 32))
        model.add(TransformerBlock(32, 2, 32))
        model.add(GlobalAveragePooling1D())
        model.add(Dropout(0.1))
        # model.add(Dense(64, activation="relu"))
        # model.add(Dropout(0.1))
        model.add(Dense(alphabet_size, activation="softmax"))
        return model

    def Trans2(bs, time_steps, alphabet_size):
        model = Sequential()
        model.add(TokenAndPositionEmbedding(time_steps, alphabet_size, 1024))
        model.add(TransformerBlock(1024, 2, 1024))
        # model.add(TransformerBlock(256, 2, 256))
        model.add(GlobalAveragePooling1D())
        model.add(Dropout(0.1))
        # model.add(Dense(128, activation="relu"))
        # model.add(Dropout(0.1))
        model.add(Dense(alphabet_size, activation="softmax"))
        return model

    def Transformer3(bs, time_steps, alphabet_size):
        model = Sequential()
        model.add(TokenAndPositionEmbedding(time_steps, alphabet_size, 32))
        model.add(TransformerBlock(32, 2, 32))
        model.add(TransformerBlock(32, 2, 32))
        model.add(TransformerBlock(32, 2, 32))
        model.add(GlobalAveragePooling1D())
        model.add(Dropout(0.1))
        #model.add(Dense(64, activation="relu"))
        #model.add(Dropout(0.1))
        model.add(Dense(alphabet_size, activation="softmax"))
        return model

    def Transformer1(bs, time_steps, alphabet_size):
        model = Sequential()
        model.add(TokenAndPositionEmbedding(time_steps, alphabet_size, 32))
        model.add(TransformerBlock(32, 2, 32))
        model.add(GlobalAveragePooling1D())
        model.add(Dropout(0.1))
        #model.add(Dense(64, activation="relu"))
        #model.add(Dropout(0.1))
        model.add(Dense(alphabet_size, activation="softmax"))
        return model

    def TransformerLarge_128(bs, time_steps, alphabet_size):
        model = Sequential()
        model.add(TokenAndPositionEmbedding(time_steps, alphabet_size, 1024))
        model.add(TransformerBlock(1024, 2, 1024))
        model.add(TransformerBlock(1024, 2, 1024))
        model.add(GlobalAveragePooling1D())
        model.add(Dropout(0.1))
        # model.add(Dense(64, activation="relu"))
        # model.add(Dropout(0.1))
        model.add(Dense(alphabet_size, activation="softmax"))
        return model


In [ ]:
#@title Create Directories and Paths
#script 

model_dir = base_path + 'data/trained_models/'
compressed_dir = base_path + 'data/compressed/'
data_dir = base_path + 'data/processed_files/'
logs_dir = base_path + 'data/logs_data/'
original_dir = base_path + 'data/files_to_be_compressed/'

try:
    os.mkdir(model_dir + base_name)
except OSError as error:
    print(error)

model_file = model_dir + base_name + '/' + model_name + '.hdf5'
log_file = logs_dir + base_name + '/' + model_name + '.log.csv'
output_dir = compressed_dir + base_name
recon_file_name = output_dir + '/' + model_name + '.reconstructed.txt'
params_file = data_dir + base_name + '.param.json'
print(params_file)
output_prefix = output_dir + '/' + model_name + '.compressed'

try:
    os.mkdir(output_dir)
except OSError as error:
    print(error)

status = recon_file_name == (original_dir + base_name + '.txt')
print(status)

try:
    os.mkdir(model_dir + base_name)
except OSError as error:
    print(error)

try:
    os.mkdir(logs_dir + base_name)
except OSError as error:
    print(error)

try:
    os.mkdir(compressed_dir + base_name)
except OSError as error:
    print(error)

In [ ]:
# @title Trainer using a custom loop optimized for TPU
#trainer
# tf.random.set_seed(42)
# np.random.seed(0)

# train_iterator = iter(train_dataset)

# with strategy.scope():
#     model = getattr(Models, model_name)(batch_size, sequence_length, alphabet_size)
#     optim = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0, amsgrad=False)
#     training_loss = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)

#     def loss_object(y_true, y_pred):
#         return 1/np.log(2) * K.categorical_crossentropy(y_true, y_pred)

# print("Starting training ...")

# @tf.function
# def train_multiple_steps(iterator, steps):
#     def step_fn(data):
#         x, y = data
#         with tf.GradientTape() as tape:
#             y_ = model(x, training=True)
#             loss = loss_object(y, y_)
#             loss_value = tf.nn.compute_average_loss(loss)

#         gradients = tape.gradient(loss_value, model.trainable_variables)
#         optim.apply_gradients(zip(gradients, model.trainable_variables))
#         training_loss.update_state(loss_value * strategy.num_replicas_in_sync)
    
#     for _ in tf.range(steps):
#         strategy.run(step_fn, args=(next(iterator),))

# train_multiple_steps(train_iterator, tf.convert_to_tensor(14))
# print('Current step: {}, training loss: {}'.format(
#       optim.iterations.numpy(),
#       round(float(training_loss.result()), 4)))

# #training loop
# for epoch in range(num_epochs):
#     with tqdm(train_dataset, unit=" batches") as tqdm_TD:
#         for data in tqdm_TD:
#             tqdm_TD.set_description(f"Epoch {epoch+1}")

#             distributed_train_step(data, 64)
#             evaluated_loss = str(training_loss.result().numpy())
#             tqdm_TD.set_postfix(Loss = evaluated_loss)
            
#     model.save_weights(model_file)
                                                            

In [ ]:
#@title Trainer using a custom loop
#trainer

# tf.random.set_seed(42)
# np.random.seed(0)
# from time import sleep

# training_data = base_path + 'data/processed_files/' + base_name + '.npy'

# print("Starting training ...")

# def loss_object(y_true, y_pred):
#     return 1/np.log(2) * K.categorical_crossentropy(y_true, y_pred)

# def loss_fn(model, x, y, training):
#     y_ = model(x, training=training)

#     return loss_object(y_true=y, y_pred=y_)

# def strided_app(a, L, S):  # Window len = L, Stride len/stepsize = S
#     nrows = ((a.size - L) // S) + 1
#     n = a.strides[0]
#     return np.lib.stride_tricks.as_strided(a, shape=(nrows, L), strides=(S * n, n), writeable=False)

# def generate_single_output_data(file_path,batch_size,time_steps):
#     series = np.load(file_path)
#     series = series.reshape(-1, 1)
#     series = series.astype('uint8')
#     print(series.itemsize, series.size)
#     onehot_encoder = OneHotEncoder(sparse=False, dtype = bool)
#     onehot_encoded = onehot_encoder.fit(series)
#     series = series.reshape(-1)

#     data = strided_app(series, time_steps+1, 1)
#     l = int(len(data)/batch_size) * batch_size
#     data = data[:l] 
    
#     X = data[:, :-1]
#     Y = data[:, -1:]
    
#     Y = onehot_encoder.transform(Y)
#     alphabet_size = Y.shape[1]

#     print("X itemsize : ", X.itemsize, "X size : ", X.size, "X shape : ", X.shape)
#     print("X memory usage in GB : ", sys.getsizeof(X)/1000000000)
#     print("Y itemsize : ", Y.itemsize, "Y size : ", Y.size, "Y shape : ", Y.shape)
#     print("Y memory usage in GB : ", sys.getsizeof(Y)/1000000000)
    
#     train_dataset = tf.data.Dataset.from_tensor_slices((X, Y))
#     train_dataset = train_dataset.batch(batch_size)
#     return train_dataset, alphabet_size

# def grad(model, inputs, targets):
#     with tf.GradientTape() as tape:
#         loss_value = loss_fn(model, inputs, targets, training=True)
#     return loss_value, tape.gradient(loss_value, model.trainable_variables)

# def fit_model(train_dataset, nb_epoch, model):
#     with strategy.scope():
#         optim = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0, amsgrad=False)
#         model.compile(loss=loss_fn, optimizer=optim, steps_per_execution = 128)

#     #training loop
#     for epoch in range(nb_epoch):
#         epoch_loss_avg = tf.keras.metrics.Mean()

#         with tqdm(train_dataset, unit=" batches") as tqdm_TD:
#             for x,y in tqdm_TD:
#                 tqdm_TD.set_description(f"Epoch {epoch+1}")

#                 loss_value, grads = grad(model, x, y)
#                 optim.apply_gradients(zip(grads, model.trainable_variables))

#                 epoch_loss_avg.update_state(loss_value)
#                 evaluated_loss = str(epoch_loss_avg.result().numpy())

#                 tqdm_TD.set_postfix(Loss = evaluated_loss)
                
#         model.save_weights(model_file)
                                                                

# #batch_size=128
# batch_size=1024
# sequence_length=64
# num_epochs=20

# train_dataset, alphabet_size = generate_single_output_data(training_data, batch_size, sequence_length)

# #print(train_dataset.element_spec)

# with strategy.scope():
#     model = getattr(Models, model_name)(batch_size, sequence_length, alphabet_size)

# print('Fitting model ...')
# fit_model(train_dataset, num_epochs, model)

In [ ]:
# @title Trainer with tf.data.Dataset - big files - quarters

# # trainer

# tf.random.set_seed(42)
# np.random.seed(0)

# training_data = base_path + 'data/processed_files/' + base_name + '.npy'

# print("Starting training ...")

# def loss_fn(y_true, y_pred):
#     #print(y_true.shape, y_pred.shape)
#     return 1/np.log(2) * K.categorical_crossentropy(y_true, y_pred)

# def strided_app(a, L, S):  # Window len = L, Stride len/stepsize = S
#     nrows = ((a.size - L) // S) + 1
#     n = a.strides[0]
#     return np.lib.stride_tricks.as_strided(a, shape=(nrows, L), strides=(S * n, n), writeable=False)

# def generate_single_output_data(file_path,batch_size,time_steps):
#     series = np.load(file_path)
#     series = series.reshape(-1, 1)
#     series = series.astype('uint8')
#     print(series.itemsize, series.size)
#     onehot_encoder = OneHotEncoder(sparse=False, dtype = 'uint8')
#     onehot_encoded = onehot_encoder.fit(series)
#     series = series.reshape(-1)

#     data = strided_app(series, time_steps+1, 1)
#     l = int(len(data)/batch_size) * batch_size
#     data = data[:l] 
    
#     X = data[:, :-1]
#     Y = data[:, -1:]
    
#     Y = onehot_encoder.transform(Y)

#     Y = np.matmul(Y,conversion_matrix)
#     # Y = Y.astype('uint8')

#     alphabet_size = Y.shape[1]

#     print("X itemsize : ", X.itemsize, "X size : ", X.size, "X shape : ", X.shape)
#     print("X memory usage in GB : ", sys.getsizeof(X)/1000000000)
#     print("Y itemsize : ", Y.itemsize, "Y size : ", Y.size, "Y shape : ", Y.shape)
#     print("Y memory usage in GB : ", sys.getsizeof(Y)/1000000000)

#     quarter = int(len(X)/4)
#     x1 = X[:quarter]
#     y1 = Y[:quarter]
#     x2 = X[quarter:2*quarter]
#     y2 = Y[quarter:2*quarter]
#     x3 = X[2*quarter:3*quarter]
#     y3 = Y[2*quarter:3*quarter]
#     x4 = X[3*quarter:]
#     y4 = Y[3*quarter:]

#     train_dataset_1 = tf.data.Dataset.from_tensor_slices((x1, y1))
#     print("Created dataset 1")
#     train_dataset_1 = train_dataset_1.batch(int(quarter/2))
#     print("Split dataset 1 into batches")

#     train_dataset_2 = tf.data.Dataset.from_tensor_slices((x2, y2))
#     print("Created dataset 2")
#     train_dataset_2 = train_dataset_2.batch(int(quarter/2))
#     print("Split dataset 2 into batches")

#     train_dataset_3 = tf.data.Dataset.from_tensor_slices((x3, y3))
#     print("Created dataset 3")
#     train_dataset_3 = train_dataset_3.batch(int(quarter/2))
#     print("Split dataset 3 into batches")

#     train_dataset_4 = tf.data.Dataset.from_tensor_slices((x2, y2))
#     print("Created dataset 4")
#     train_dataset_4 = train_dataset_4.batch(int(quarter/2))
#     print("Split dataset 4 into batches")

#     return train_dataset_1, train_dataset_2, train_dataset_3, train_dataset_4, alphabet_size
    

# def fit_model(train_1, train_2, train_3, train_4, bs, nb_epoch, model):
#     with strategy.scope():
#         optim = tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0, amsgrad=False)
#         model.compile(loss=loss_fn, optimizer=optim, steps_per_execution = 128)

#     checkpoint = ModelCheckpoint(model_file, monitor='loss', verbose=1, save_best_only=False, mode='min', save_weights_only=True)
#     csv_logger = CSVLogger(log_file, append=True, separator=';')
#     #early_stopping = EarlyStopping(monitor='loss', mode='min', min_delta=0.005, patience=3, verbose=1)
#     callbacks_list = [checkpoint, csv_logger]
#     for i in range(nb_epoch):
#         print("Epoch: ", i)
#         train_1 = train_1.shuffle(buffer_size = len(train_1))
#         train_2 = train_2.shuffle(buffer_size = len(train_2))
#         train_3 = train_3.shuffle(buffer_size = len(train_3))
#         train_4 = train_4.shuffle(buffer_size = len(train_4))

#         j = 1
#         for x,y in train_1:
#             print("Minibatch number", j)
#             model.fit(x, y, epochs=1, batch_size=bs, verbose=1, callbacks=callbacks_list)
#             j = j + 1
        
#         for x,y in train_2:
#             print("Minibatch number", j)
#             model.fit(x, y, epochs=1, batch_size=bs, verbose=1, callbacks=callbacks_list)
#             j = j + 1

#         for x,y in train_3:
#             print("Minibatch number", j)
#             model.fit(x, y, epochs=1, batch_size=bs, verbose=1, callbacks=callbacks_list)
#             j = j + 1

#         for x,y in train_4:
#             print("Minibatch number", j)
#             model.fit(x, y, epochs=1, batch_size=bs, verbose=1, callbacks=callbacks_list)
#             j = j + 1
        
# #batch_size=128
# batch_size=1024
# sequence_length=64
# num_epochs=5

# training_data_1, training_data_2, training_data_3, training_data_4, alphabet_size = generate_single_output_data(training_data, batch_size, sequence_length)

# with strategy.scope():
#     model = getattr(Models, model_name)(batch_size, sequence_length, alphabet_size)
# print('Fitting model ...')
# fit_model(training_data_1, training_data_2, training_data_3, training_data_4, batch_size, num_epochs, model)


In [ ]:
#@title Trainer with tf.data.Dataset - halves

#trainer

# tf.random.set_seed(42)
# np.random.seed(0)

# training_data = base_path + 'data/processed_files/' + base_name + '.npy'

# print("Starting training ...")

# def loss_fn(y_true, y_pred):
#     #print(y_true.shape, y_pred.shape)
#     return 1/np.log(2) * K.categorical_crossentropy(y_true, y_pred)

# def strided_app(a, L, S):  # Window len = L, Stride len/stepsize = S
#     nrows = ((a.size - L) // S) + 1
#     n = a.strides[0]
#     return np.lib.stride_tricks.as_strided(a, shape=(nrows, L), strides=(S * n, n), writeable=False)

# def generate_single_output_data(file_path,batch_size,time_steps):
#     series = np.load(file_path)
#     series = series.reshape(-1, 1)
#     series = series.astype('uint8')
#     print(series.itemsize, series.size)
#     onehot_encoder = OneHotEncoder(sparse=False, dtype = 'uint8')
#     onehot_encoded = onehot_encoder.fit(series)
#     series = series.reshape(-1)

#     data = strided_app(series, time_steps+1, 1)
#     l = int(len(data)/batch_size) * batch_size
#     data = data[:l] 
    
#     X = data[:, :-1]
#     Y = data[:, -1:]
    
#     Y = onehot_encoder.transform(Y)
    
#     Y = np.matmul(Y,conversion_matrix)
#     Y = Y.astype('uint8')

#     alphabet_size = Y.shape[1]

#     print("X itemsize : ", X.itemsize, "X size : ", X.size, "X shape : ", X.shape)
#     print("X memory usage in GB : ", sys.getsizeof(X)/1000000000)
#     print("Y itemsize : ", Y.itemsize, "Y size : ", Y.size, "Y shape : ", Y.shape)
#     print("Y memory usage in GB : ", sys.getsizeof(Y)/1000000000)

#     half = int(len(X)/2)
#     x1 = X[:half]
#     y1 = Y[:half]
#     x2 = X[half:]
#     y2 = Y[half:]

#     train_dataset_1 = tf.data.Dataset.from_tensor_slices((x1, y1))
#     print("Created dataset 1")
#     train_dataset_1 = train_dataset_1.batch(int(half/2))
#     print("Split dataset 1 into batches")

#     train_dataset_2 = tf.data.Dataset.from_tensor_slices((x2, y2))
#     print("Created dataset 2")
#     train_dataset_2 = train_dataset_2.batch(int(half/2))
#     print("Split dataset 2 into batches")

#     return train_dataset_1, train_dataset_2, alphabet_size
    

# def fit_model(train_1, train_2, bs, nb_epoch, model):
#     with strategy.scope():
#         optim = tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0, amsgrad=False)
#         model.compile(loss=loss_fn, optimizer=optim, steps_per_execution = 128)

#     checkpoint = ModelCheckpoint(model_file, monitor='loss', verbose=1, save_best_only=False, mode='min', save_weights_only=True)
#     csv_logger = CSVLogger(log_file, append=True, separator=';')
#     early_stopping = EarlyStopping(monitor='loss', mode='min', min_delta=0.005, patience=3, verbose=1)
#     callbacks_list = [checkpoint, csv_logger]
#     for i in range(nb_epoch):
#         train_1 = train_1.shuffle(buffer_size = len(train_1))
#         train_2 = train_2.shuffle(buffer_size = len(train_2))

#         j = 1
#         for x,y in train_1:
#             print("Minibatch number", j)
#             model.fit(x, y, epochs=i+2, initial_epoch=i, batch_size=bs, verbose=1, callbacks=callbacks_list)
#             j = j + 1
        
#         for x,y in train_2:
#             print("Minibatch number", j)
#             model.fit(x, y, epochs=i+2, initial_epoch=i, batch_size=bs, verbose=1, callbacks=callbacks_list)
#             j = j + 1
        
# #batch_size=128
# batch_size=1024
# sequence_length=64
# num_epochs=5

# training_data_1, training_data_2, alphabet_size = generate_single_output_data(training_data, batch_size, sequence_length)

# with strategy.scope():
#     model = getattr(Models, model_name)(batch_size, sequence_length, alphabet_size)
# print('Fitting model ...')
# fit_model(training_data_1, training_data_2, batch_size, num_epochs, model)

In [ ]:
#@title Trainer without data.Dataset
#trainer

tf.random.set_seed(42)
np.random.seed(0)

training_data = base_path + 'data/processed_files/' + base_name + '.npy'

print("Starting training ...")

def loss_fn(y_true, y_pred):
    #print(y_true.shape, y_pred.shape)
    return 1/np.log(2) * K.categorical_crossentropy(y_true, y_pred)

def strided_app(a, L, S):  # Window len = L, Stride len/stepsize = S
    nrows = ((a.size - L) // S) + 1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows, L), strides=(S * n, n), writeable=False)

def generate_single_output_data(file_path,batch_size,time_steps):
    series = np.load(file_path)
    series = series.reshape(-1, 1)
    series = series.astype('uint8')
    print(series.itemsize, series.size)
    onehot_encoder = OneHotEncoder(sparse=False, dtype = 'uint8')
    onehot_encoded = onehot_encoder.fit(series)
    series = series.reshape(-1)

    data = strided_app(series, time_steps+1, 1)
    l = int(len(data)/batch_size) * batch_size
    data = data[:l] 
    
    X = data[:, :-1]
    Y = data[:, -1:]
    
    Y = onehot_encoder.transform(Y)

    Y = np.matmul(Y,conversion_matrix)
    Y = Y.astype('uint8')
    
    alphabet_size = Y.shape[1]

    print("X itemsize : ", X.itemsize, "X size : ", X.size, "X shape : ", X.shape)
    print("X memory usage in GB : ", sys.getsizeof(X)/1000000000)
    print("Y itemsize : ", Y.itemsize, "Y size : ", Y.size, "Y shape : ", Y.shape)
    print("Y memory usage in GB : ", sys.getsizeof(Y)/1000000000)

    return X, Y, alphabet_size
    

def fit_model(x, y, bs, nb_epoch, model):
    with strategy.scope():
        optim = tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0, amsgrad=False)
        model.compile(loss=loss_fn, optimizer=optim, steps_per_execution = 128)

    checkpoint = ModelCheckpoint(model_file, monitor='loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)
    csv_logger = CSVLogger(log_file, append=True, separator=';')
    early_stopping = EarlyStopping(monitor='loss', mode='min', min_delta=0.005, patience=3, verbose=1)
    callbacks_list = [checkpoint, csv_logger, early_stopping]
    
    model.fit(x, y, epochs=nb_epoch, shuffle=True, batch_size=bs, verbose=1, callbacks=callbacks_list)

batch_size=1024
sequence_length=64
num_epochs=20

X, Y, alphabet_size = generate_single_output_data(training_data, batch_size, sequence_length)

with strategy.scope():
    model = getattr(Models, model_name)(batch_size, sequence_length, alphabet_size)
print('Fitting model ...')
fit_model(X, Y, batch_size, num_epochs, model)

Starting training ...
1 465559
X itemsize :  1 X size :  29753344 X shape :  (464896, 64)
X memory usage in GB :  1.2e-07
Y itemsize :  1 Y size :  59506688 Y shape :  (464896, 128)
Y memory usage in GB :  0.059506808
Fitting model ...
Epoch 1/20
454/454 [==============================] - ETA: 0s - loss: 4.3541
Epoch 1: loss improved from inf to 4.35409, saving model to /content/drive/MyDrive/DeepZip/data/trained_models/HP1_clean/TransformerLarge_128.hdf5
454/454 [==============================] - 47s 104ms/step - loss: 4.3541
Epoch 2/20
454/454 [==============================] - ETA: 0s - loss: 3.5351
Epoch 2: loss improved from 4.35409 to 3.53508, saving model to /content/drive/MyDrive/DeepZip/data/trained_models/HP1_clean/TransformerLarge_128.hdf5
454/454 [==============================] - 29s 64ms/step - loss: 3.5351
Epoch 3/20
454/454 [==============================] - ETA: 0s - loss: 3.2934
Epoch 3: loss improved from 3.53508 to 3.29340, saving model to /content/drive/MyDrive/Dee

In [ ]:
#@title Compressor

print('Starting Compression ...')

def strided_app(a, L, S):
    nrows = ((a.size - L) // S) + 1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows, L), strides=(S * n, n), writeable=False)

def predict_lstm(X, y, y_original, timesteps, bs, alphabet_size, model_name, final_step=False):
    with strategy.scope():
        model = getattr(Models, model_name)(bs, timesteps, alphabet_size)
    
    model(np.zeros(shape=(128,64), dtype = np.uint8))
    model.load_weights(model_file)

    if not final_step:
        num_iters = int((len(X)+timesteps)/bs)
        ind = np.array(range(bs))*num_iters
        
        f = [open(temp_file_prefix + '.' + str(i),'wb') for i in range(bs)]
        bitout = [BitOutputStream(f[i]) for i in range(bs)]
        enc = [ArithmeticEncoder(32, bitout[i]) for i in range(bs)]
        prob = np.ones(alphabet_size)/alphabet_size
        cumul = np.zeros(alphabet_size + 1, dtype = np.uint64)
        cumul[1:] = np.cumsum(prob*10000000 + 1)    

        for i in range(bs):
            for j in range(min(timesteps, num_iters)):
                enc[i].write(cumul, X[ind[i],j])

        cumul = np.zeros((bs, alphabet_size+1), dtype = np.uint64)
        
        for j in tqdm(range(num_iters - timesteps)):
            prob = model.predict(X[ind,:], batch_size=bs)
            cumul[:,1:] = np.cumsum(prob*10000000 + 1, axis = 1)

            for i in range(bs):
                enc[i].write(cumul[i,:], y_original[ind[i]])

            ind = ind + 1

        # close files
        for i in range(bs):
            enc[i].finish()
            bitout[i].close()
            f[i].close()            
    else:
        f = open(temp_file_prefix + '.last', 'wb')
        bitout = BitOutputStream(f)
        enc = ArithmeticEncoder(32, bitout)
        prob = np.ones(alphabet_size)/alphabet_size
        cumul = np.zeros(alphabet_size+1, dtype = np.uint64)
        cumul[1:] = np.cumsum(prob*10000000 + 1)        

        for j in range(timesteps):
            enc.write(cumul, X[0,j])

        for i in tqdm(range(len(X))):
            prob = model.predict(X[i,:].reshape(1,-1), batch_size=1)
            cumul[1:] = np.cumsum(prob*10000000 + 1)
            enc.write(cumul, y_original[i][0])

        enc.finish()
        bitout.close()
        f.close()
    return

def var_int_encode(byte_str_len, f):
    while True:
        this_byte = byte_str_len&127
        byte_str_len >>= 7
        if byte_str_len == 0:
            f.write(struct.pack('B',this_byte))
            break

        f.write(struct.pack('B',this_byte|128))
        byte_str_len -= 1

#main
temp_dir = tempfile.mkdtemp()
temp_file_prefix = temp_dir + "/compressed"

tf.random.set_seed(42)
np.random.seed(0)
series = np.load(training_data)
series = series.reshape(-1, 1)
onehot_encoder = OneHotEncoder(sparse=False, dtype = bool)
onehot_encoded = onehot_encoder.fit(series)

print(len(series))
batch_size = 1024

timesteps = 64


with open(params_file, 'r') as f:
    params = json.load(f)

params['len_series'] = len(series)
params['bs'] = batch_size
params['timesteps'] = timesteps

with open(output_prefix + '.params', 'w') as f:
    json.dump(params, f, indent=4)

alphabet_size = len(params['id2char_dict'])

series = series.reshape(-1)
data = strided_app(series, timesteps+1, 1)

X = data[:, :-1]
Y_original = data[:, -1:]

X = X.astype('uint8')
Y_original = Y_original.astype('uint8')
Y = onehot_encoder.transform(Y_original)

print("X itemsize : ", X.itemsize, "X size : ", X.size, "X shape : ", X.shape)
print("X memory usage in GB : ", sys.getsizeof(X)/1000000000)
print("Y itemsize : ", Y.itemsize, "Y size : ", Y.size, "Y shape : ", Y.shape)
print("Y memory usage in GB : ", sys.getsizeof(Y)/1000000000)

l = int(len(series)/batch_size)*batch_size

with strategy.scope():
    predict_lstm(X, Y, Y_original, timesteps, batch_size, alphabet_size, model_name)

if l < len(series)-timesteps:
    with strategy.scope():
        predict_lstm(X[l:,:], Y[l:,:], Y_original[l:], timesteps, 1, alphabet_size, model_name, final_step = True)
else:
    f = open(temp_file_prefix + '.last', 'wb')
    bitout = BitOutputStream(f)
    enc = ArithmeticEncoder(32, bitout) 
    prob = np.ones(alphabet_size)/alphabet_size
    
    cumul = np.zeros(alphabet_size+1, dtype = np.uint64)
    cumul[1:] = np.cumsum(prob*10000000 + 1)        
    for j in range(l, len(series)):
        enc.write(cumul, series[j])

    enc.finish()
    bitout.close() 
    f.close()

f = open(output_prefix + '.combined', 'wb')
for i in range(batch_size):
    f_in = open(temp_file_prefix + '.' + str(i), 'rb')
    byte_str = f_in.read()
    byte_str_len = len(byte_str)
    var_int_encode(byte_str_len, f)
    f.write(byte_str)
    f_in.close()

f_in = open(temp_file_prefix + '.last', 'rb')
byte_str = f_in.read()
byte_str_len = len(byte_str)
var_int_encode(byte_str_len, f)
f.write(byte_str)
f_in.close()
f.close()
shutil.rmtree(temp_dir)

In [ ]:
#@title Decompressor

# input_file_prefix = output_dir + '/' + model_name + '.compressed'
# print(input_file_prefix)

# def strided_app(a, L, S):
#     nrows = ((a.size - L) // S) + 1
#     n = a.strides[0]
#     return np.lib.stride_tricks.as_strided(a, shape=(nrows, L), strides=(S * n, n), writeable=False)

# def create_data(rows, p=0.5):
#     data = np.random.choice(2, rows, p=[p, 1-p])
#     print(np.sum(data))/np.float32(len(data))
#     return data
 
# def predict_lstm(len_series, timesteps, bs, alphabet_size, model_name, final_step=False):
#     with strategy.scope():
#         model = getattr(Models, model_name)(bs, timesteps, alphabet_size)
#     model(np.zeros(shape=(128,64), dtype = np.uint8))
#     model.load_weights(model_file)
    
#     if not final_step:
#         num_iters = int((len_series)/bs)
#         series_2d = np.zeros((bs,num_iters), dtype = np.uint8)
#         f = [open(temp_file_prefix+'.'+str(i),'rb') for i in range(bs)]
#         bitin = [BitInputStream(f[i]) for i in range(bs)]
#         dec = [ArithmeticDecoder(32, bitin[i]) for i in range(bs)]
#         prob = np.ones(alphabet_size)/alphabet_size
#         cumul = np.zeros(alphabet_size+1, dtype = np.uint64)
#         cumul[1:] = np.cumsum(prob*10000000 + 1)              

#         for i in range(bs):
#             for j in range(min(num_iters,timesteps)):
#                 series_2d[i,j] = dec[i].read(cumul, alphabet_size)

#         cumul = np.zeros((bs, alphabet_size+1), dtype = np.uint64)

#         for j in tqdm(range(num_iters - timesteps)):
#             prob = model.predict(series_2d[:,j:j+timesteps], batch_size=bs)
#             cumul[:,1:] = np.cumsum(prob*10000000 + 1, axis = 1)
#             for i in range(bs):
#                 series_2d[i,j+timesteps] = dec[i].read(cumul[i,:], alphabet_size)

#         for i in range(bs):
#             bitin[i].close()
#             f[i].close()

#         return series_2d.reshape(-1)

#     else:
#         series = np.zeros(len_series, dtype = np.uint8)
#         f = open(temp_file_prefix + '.last', 'rb')
#         bitin = BitInputStream(f)
#         dec = ArithmeticDecoder(32, bitin)
#         prob = np.ones(alphabet_size)/alphabet_size

#         cumul = np.zeros(alphabet_size+1, dtype = np.uint64)
#         cumul[1:] = np.cumsum(prob*10000000 + 1)                
#         for j in range(min(timesteps,len_series)):
#             series[j] = dec.read(cumul, alphabet_size)
            
#         for i in tqdm(range(len_series-timesteps)):
#             prob = model.predict(series[i:i+timesteps].reshape(1,-1), batch_size=1)
#             cumul[1:] = np.cumsum(prob*10000000 + 1)
#             series[i+timesteps] = dec.read(cumul, alphabet_size)
#         bitin.close()
#         f.close()
#         return series

# def arithmetic_step(prob, freqs, dec):
#     freqs.update_table(prob*10000000+1)
#     return dec.read(freqs)

# def var_int_decode(f):
#     byte_str_len = 0
#     shift = 1
#     while True:
#         this_byte = struct.unpack('B', f.read(1))[0]
#         byte_str_len += (this_byte & 127) * shift
#         if this_byte & 128 == 0:
#             break
#         shift <<= 7
#         byte_str_len += shift
#     return byte_str_len

# #main
# temp_dir = tempfile.mkdtemp()
# temp_file_prefix = temp_dir + "/compressed"

# tf.random.set_seed(42)
# np.random.seed(0)

# f = open(input_file_prefix + '.params','r')
# param_dict = json.loads(f.read())
# f.close()
# len_series = param_dict['len_series']
# batch_size = param_dict['bs']
# timesteps = param_dict['timesteps']
# id2char_dict = param_dict['id2char_dict']

# f = open(input_file_prefix+'.combined','rb')

# for i in range(batch_size):
#     f_out = open(temp_file_prefix+'.'+str(i),'wb')
#     byte_str_len = var_int_decode(f)
#     byte_str = f.read(byte_str_len)
#     f_out.write(byte_str)
#     f_out.close()

# f_out = open(temp_file_prefix+'.last','wb')
# byte_str_len = var_int_decode(f)
# byte_str = f.read(byte_str_len)
# f_out.write(byte_str)
# f_out.close()
# f.close()

# series = np.zeros(len_series,dtype=np.uint8)

# l = int(len_series/batch_size)*batch_size
# alphabet_size = len(id2char_dict)
# with strategy.scope():
#     series[:l] = predict_lstm(l, timesteps, batch_size, alphabet_size, model_name)

# if l < len_series:
#     with strategy.scope():
#         series[l:] = predict_lstm(len_series - l, timesteps, 1, alphabet_size, model_name, final_step = True)

# f = open(recon_file_name,'wb')
# print(id2char_dict)
# print(series[:10])
# f.write(bytearray([id2char_dict[str(s)] for s in series]))
# f.close()
# shutil.rmtree(temp_dir)

In [ ]:
#@title Results
import filecmp

compressed_file_path = input_file_prefix + '.combined'

model_size = os.stat(model_file).st_size
compressed_size = os.stat(compressed_file_path).st_size + model_size
original_size = os.stat(input_file_path).st_size
# reconstructed_size = os.stat(recon_file_name).st_size

print(f'Original file size is {original_size} bytes')
print(f'Compressed file size is {compressed_size} bytes')
# print(f'Reconstructed file size is {reconstructed_size} bytes')
print(f'Model size is {model_size} bytes')

# diff = filecmp.cmp(input_file_path, recon_file_name)
# if diff:
#     print("Reconstructed file is identical to original file")
# else:
#     print("Reconstructed file is not identical to original file")

In [ ]:
def model_size(m, x, a):
    return 8*(m*(9*x*x + 13*x) + a*x + a)

m = 1
a = 200
x = 1024

print(model_size(m,x,a)/1000000)